In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from pathlib import Path

In [2]:
from diskcache import Cache

cache = Cache("matches.shelve")

@cache.memoize()
def get_page(link):
    print(link)
    return requests.get(link).content

In [3]:
list(cache.iterkeys())

[('__main__.get_page', 'https://understat.com/league/EPL/2014'),
 ('__main__.get_page', 'https://understat.com/league/EPL/2015'),
 ('__main__.get_page', 'https://understat.com/league/EPL/2016'),
 ('__main__.get_page', 'https://understat.com/league/EPL/2017'),
 ('__main__.get_page', 'https://understat.com/league/EPL/2018'),
 ('__main__.get_page', 'https://understat.com/league/EPL/2019'),
 ('__main__.get_page', 'https://understat.com/league/EPL/2020'),
 ('__main__.get_page', 'https://understat.com/league/La_liga/2014'),
 ('__main__.get_page', 'https://understat.com/league/La_liga/2015'),
 ('__main__.get_page', 'https://understat.com/league/La_liga/2016'),
 ('__main__.get_page', 'https://understat.com/league/La_liga/2017'),
 ('__main__.get_page', 'https://understat.com/league/La_liga/2018'),
 ('__main__.get_page', 'https://understat.com/league/La_liga/2019'),
 ('__main__.get_page', 'https://understat.com/league/La_liga/2020'),
 ('__main__.get_page', 'https://understat.com/league/Ligue_1/2

In [4]:
import sys
import collections

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [5]:
leagues = ['EPL', 'La_liga', 'Bundesliga', 'Serie_A', 'Ligue_1']
years = {
    '2020-2021': '2020',
    '2019-2020': '2019',
    '2018-2019': '2018',
    '2017-2018': '2017',
    '2016-2017': '2016',
    '2015-2016': '2015',
    '2014-2015': '2014',
}

In [6]:
dfs = []
for league in leagues:
    for season in years:
        year = years[season]
        url = f'https://understat.com/league/{league}/{year}'
        
        res = get_page(url)
        soup = BeautifulSoup(res, 'lxml')
        scripts = soup.find_all('script')
        strings = scripts[1].string
        
        ind_start = strings.index("('")+2 
        ind_end = strings.index("')") 
        json_data = strings[ind_start:ind_end] 
        json_data = json_data.encode('utf8').decode('unicode_escape')
        
        data = json.loads(json_data)
        
        data = [flatten(d) for d in data]
        
        df = pd.DataFrame(data)
        df['season'] = season
        df['league'] = league
        df['datetime'] = pd.to_datetime(df['datetime'])
        df = df.sort_values(by = 'datetime', ascending = False)
        dfs.append(df)
    
df = pd.concat(dfs, ignore_index= True)

path = Path(f'dfs')
path.mkdir(parents=True, exist_ok=True)

df.to_csv(os.path.join(path, 'matches.csv'), index = False)

/opt/anaconda3/envs/general/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  


In [7]:
df['league'].unique()

array(['EPL', 'La_liga', 'Bundesliga', 'Serie_A', 'Ligue_1'], dtype=object)

In [8]:
len(df)

12782